In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
with open("hp_parsed.json", "r") as f:
    result = json.load(f)


dfs_ebs = {}
for key in ["components", "connections"]:
    dfs_ebs[key] = pd.DataFrame.from_dict(result[key], orient="index").sort_index()

In [3]:
with open("hp_tespy.json", "r") as f:
    result = json.load(f)


dfs_tespy = {}
for key in ["components", "connections"]:
    dfs_tespy[key] = pd.DataFrame.from_dict(result[key], orient="index").sort_index()

In [4]:
dfs_ebs["connections"][["e_T", "e_M"]] = dfs_ebs["connections"][["e_T", "e_M"]].round(6)
dfs_tespy["connections"][["e_T", "e_M"]] = dfs_tespy["connections"][["e_T", "e_M"]].round(6)

In [5]:
columns = ["m", "p", "T", "e_T", "e_M"]

overlapping_index = list(set(dfs_tespy[key].index.tolist()) & set(dfs_ebs[key].index.tolist()))
diff_to_tespy = (
    (
        dfs_ebs[key].loc[overlapping_index, columns]
        - dfs_tespy[key].loc[overlapping_index, columns]
    ) / dfs_tespy[key].loc[overlapping_index, columns]
).abs().replace(np.inf, np.nan)
diff_to_ebs = (
    (
        dfs_ebs[key].loc[overlapping_index, columns]
        - dfs_tespy[key].loc[overlapping_index, columns]
    ).abs() / dfs_tespy[key].loc[overlapping_index, columns]
).abs().replace(np.inf, np.nan)

In [6]:
diff_to_ebs

,m,p,T,e_T,e_M
13,0.000002,0.000000e+00,3.415439e-10,2.660484e-04,NaN
30,0.000079,0.000000e+00,2.481873e-09,1.481377e-04,0.000000e+00
32,0.000079,0.000000e+00,5.074101e-09,1.274528e-04,2.583452e-07
33,0.000079,1.459999e-10,1.154451e-05,2.720777e-03,2.157887e-07
31,0.000079,0.000000e+00,2.107637e-09,1.481416e-04,0.000000e+00
34,0.000079,0.000000e+00,3.992981e-05,6.342950e-05,0.000000e+00
22,0.000000,0.000000e+00,2.444620e-09,1.420389e-06,4.665026e-04
21,0.000000,0.000000e+00,5.585294e-10,1.440545e-06,4.723624e-04
12,0.000002,9.975535e-10,1.134165e-05,2.645430e-03,2.433095e-04
11,0.000002,0.000000e+00,4.416582e-14,NaN,NaN


In [7]:
diff_to_ebs[diff_to_ebs > 1e-3].dropna(how="all")

,m,p,T,e_T,e_M
33,NaN,NaN,NaN,0.002721,NaN
12,NaN,NaN,NaN,0.002645,NaN
